# GANCA Results Visualization


## Snapshot to Voxel

Code used for turning snapshots in the generation into voxel plots

In [15]:
# import stuff

import data_helper
from data_helper import GANCA3DDataModule
from models import VoxelPerceptionNet, VoxelUpdateNet, VoxelNCAModel, VoxelDiscriminator
import visualise_helper
import utils
%load_ext autoreload
from tqdm.notebook import tqdm
%autoreload 2

import numpy as np
import sys
import torch
import random
from torchsummaryX import summary
import matplotlib.pyplot as plt
from pytorch_lightning import Trainer
import pytorch_lightning as pl
import math

from loguru import logger as gurulogger
gurulogger.remove()
gurulogger.add(sys.stdout, colorize=True, format="<blue>{time}</blue> <level>{message}</level>")
gurulogger.level("INFO", color="<red><bold>")

import os
from einops import rearrange
import torch.nn.functional as F
import pandas as pd

BLOCK2VEC_OUT_PATH = 'output/block2vec saves/block2vec 64 dim locked air/'
NUM_WORKERS = int(os.cpu_count() / 2)

plt.ioff()
%matplotlib agg
%matplotlib agg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
embedding, mcid2block, block2embeddingidx, embeddingidx2block, block2mcid = utils.get_embedding_info(BLOCK2VEC_OUT_PATH)
converter = utils.DataConverter(embedding, mcid2block, block2embeddingidx, embeddingidx2block, block2mcid)

In [19]:
max_epoch = 10
snapshots_folder_path = 'lightning_logs/GANCA/version_15/gen_snapshots'
n_cols = 4

### Logged progress per epoch to image

In [ ]:
for i in range(max_epoch):
    loaded = np.load(snapshots_folder_path + f'/epoch_{i}.npy')
    fig = visualise_helper.states_to_graphs(
            loaded, 
            embedding.weight.cpu(), 
            n_cols=n_cols, 
            n_rows= int(loaded.shape[0] / n_cols), 
            converter_class=converter, 
            size_multiplier=4,
            explode=True,
            trim=True,
            no_air = True,
        )
    fig.suptitle(f'visualizing epoch {i}')
    fig.savefig(snapshots_folder_path + f'/epoch_{i}.png')

### Visualising alpha channel

In [ ]:
for i in range(max_epoch):
    loaded = np.load(snapshots_folder_path + f'/epoch_{i}_alpha.npy')
    fig = visualise_helper.alpha_states_to_graphs(
            loaded, 
            n_cols=n_cols, 
            n_rows= int(loaded.shape[0] / n_cols), 
            size_multiplier=4,
            explode = True, # this makes covered voxels visible
            trim = True, # remove uselsss white info
        )
    fig.suptitle(f'visualizing epoch {i}')
    fig.savefig(snapshots_folder_path + f'/epoch_{i}_alpha.png')

## Metrics Visualise

In [ ]:
# load from tensorboard log